# Классификация: IC50 > медианы

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Загрузка данных
df = pd.read_csv('/Users/rem/МИФИ/курсовая/dataset.csv')

# Подготовка данных: исключаем столбцы с целевыми переменными
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
X = df[features]
y = (df["IC50, mM"] > df['IC50, mM'].median()).astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Для моделей на основе градиентного бустинга (LightGBM) необходимо очистить имена признаков от специальных символов
def clean_column(name):
    return re.sub(r'[^\w]', '_', name)

X_train_clean = X_train.rename(columns=lambda col: clean_column(col))
X_test_clean = X_test.rename(columns=lambda col: clean_column(col))

# Обучение моделей
logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)
y_pred_logit = logit.predict(X_test_scaled)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train_clean, y_train)
y_pred_lgb = lgb.predict(X_test_clean)

for name, preds, clf, xtest in [("Logistic Regression", y_pred_logit, logit, X_test_scaled),
                                ("Random Forest", y_pred_rf, rf, X_test),
                                ("XGBoost", y_pred_xgb, xgb, X_test),
                                ("LightGBM", y_pred_lgb, lgb, X_test)]:
    print(f"{name}: Accuracy={accuracy_score(y_test, preds):.3f}, Precision={precision_score(y_test, preds):.3f}, Recall={recall_score(y_test, preds):.3f}, F1={f1_score(y_test, preds):.3f}, AUC={roc_auc_score(y_test, clf.predict_proba(xtest)[:, 1]):.3f}")


/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:16:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 386, number of negative: 386
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003867 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18249
[LightGBM] [Info] Number of data points in the train set: 772, number of used features: 169
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

1. Logistic Regression

Accuracy = 0.727: Модель правильно классифицирует 72,7% объектов.
Precision = 0.724: Из предсказанных положительных случаев около 72,4% действительно положительные.
Recall = 0.732: Модель находит примерно 73,2% всех реальных положительных примеров.
F1 = 0.728: Сбалансированное значение, подтверждающее умеренное качество модели по обоим аспектам — точности и полноте.
AUC = 0.806: Довольно высокий показатель дискриминационной способности модели, т.е. она хорошо различает классы при изменении порога.
Вывод: Logistic Regression демонстрирует стабильные и сбалансированные показатели, показывая, что при данном пороге модель работает на уровне около 73% точности, а AUC свидетельствует о хорошем различении классов.

2. Random Forest

Accuracy = 0.727: Тот же уровень общей точности, что и у логистической регрессии (72,7%).
Precision = 0.729: Чуть выше, чем у Logistic Regression — около 72,9% предсказанных положительных действительно положительны.
Recall = 0.722: Чуть ниже (72,2%), что означает незначительное снижение способности обнаруживать все положительные случаи по сравнению с Logistic Regression.
F1 = 0.725: Схожее значение, отражающее баланс между precision и recall.
AUC = 0.802: Немного ниже, чем у логистической регрессии, но все равно на высоком уровне.
Вывод: Random Forest показывает результаты, практически идентичные Logistic Regression по общей точности и сбалансированным метрикам, с незначительными преимуществами по precision, но немного уступая по recall и AUC.

3. XGBoost

Accuracy = 0.747: Наивысшая точность среди рассмотренных моделей — 74,7% объектов классифицируются правильно.
Precision = 0.755: Более высокая точность, чем в предыдущих моделях (75,5% предсказанных положительных являются истинными положительными).
Recall = 0.732: Остается на уровне Logistic Regression (73,2%), что указывает на схожую способность обнаруживать все положительные случаи.
F1 = 0.743: Сбалансированное значение, указывающее на общее улучшение по сравнению с Logistic Regression и Random Forest.
AUC = 0.791: Значение AUC несколько ниже, чем у Logistic Regression и Random Forest, что может говорить о том, что способность модели различать классы при изменении порога оказалась чуть менее выраженной.
Вывод: XGBoost демонстрирует наивысшую точность, precision и F1-score, что указывает на лучшее качество классификации при фиксированном пороге. Однако несколько более низкое AUC говорит о том, что при варьировании порога его способность различать классы немного уступает другим моделям.

4. LightGBM

Accuracy = 0.711: Общая точность составляет 71,1%, что ниже, чем у остальных моделей.
Precision = 0.716: Значение precision составляет 71,6% — ниже показатели Logistic Regression и Random Forest.
Recall = 0.701: Полнота ниже (70,1%), что указывает на то, что модель пропускает больше реальных положительных случаев.
F1 = 0.708: Итоговый сбалансированный показатель также ниже.
AUC = 0.802: При этом способность различать классы по AUC (0.802) остаётся на уровне Random Forest.
Вывод: Хотя LightGBM показывает конкурентоспособное значение AUC, его значения точности, precision, recall и F1-score оказались ниже, что указывает на более слабую классификационную способность при выбранном пороге.

Общая интерпретация и рекомендации

Сравнение по общей точности и сбалансированным метрикам:

XGBoost лидирует по accuracy (0.747), precision (0.755) и F1 (0.743). Это говорит о том, что при фиксированном пороге XGBoost лучше всего справляется с классификацией объектов.
Logistic Regression и Random Forest показывают очень схожие результаты (accuracy ≈ 0.727), что свидетельствует о стабильности базовой модели и ансамблевого подхода.
LightGBM отстаёт по основным метрикам (accuracy 0.711, F1 0.708), несмотря на хороший AUC, что может означать, что оптимизация порога или гиперпараметров может улучшить его результаты.
Анализ AUC:

Модель Logistic Regression демонстрирует лучший AUC (0.806), что указывает на высокую способность правильно ранжировать объекты по вероятности принадлежности к положительному классу.
Random Forest и LightGBM также показывают высокий AUC (около 0.802), тогда как XGBoost немного отстаёт (0.791). Это может свидетельствовать о том, что, несмотря на высокую классификационную точность XGBoost при выбранном пороге, его способность различать классы при изменении порога несколько ниже.
Вывод по выбору модели:

Если основная цель — максимизировать точность при фиксированном пороге, то XGBoost представляется предпочтительным выбором.
Если важна интерпретация вероятностей и способность различать классы (например, для построения ROC-кривой или дальнейшей калибровки), то Logistic Regression или Random Forest могут быть более выгодными.
LightGBM в данном случае показывает несколько менее удовлетворительные результаты, поэтому для его применения потребуется дополнительная настройка.
Таким образом, все модели демонстрируют свою силу в зависимости от выбранных приоритетов (точность vs. дискриминация по AUC), и окончательный выбор модели может зависеть от специфики задачи, требований к интерпретируемости и дальнейшего использования предсказанных вероятностей.